In [1]:
"""
Enterprise Conversational RAG System
- Low-latency (<15s on CPU)
- Modular design for easy component replacement
- Comprehensive logging for performance monitoring
- Hybrid retrieval (BM25 + Embeddings)
"""

import os
import time
import logging
import json
from typing import List, Dict, Any, Tuple, Optional
from dataclasses import dataclass
import torch
from tqdm import tqdm

import os
import psutil
import multiprocessing
from functools import lru_cache

In [2]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("rag_system.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("rag_system")

In [3]:
# Performance tracking
@dataclass
class LatencyTracker:
    start_time: float = 0
    
    def start(self):
        self.start_time = time.time()
        return self
    
    def end(self, operation_name: str) -> float:
        duration = time.time() - self.start_time
        logger.info(f"LATENCY: {operation_name} took {duration:.4f} seconds")
        return duration

##### Document Processing

In [4]:
class DocumentProcessor:
    """Handles document loading and extraction from various formats"""
    
    def __init__(self):
        try:
            from unstructured.partition.pdf import partition_pdf
            self.partition_pdf = partition_pdf
        except ImportError:
            logger.error("unstructured package not installed. Install with: pip install unstructured pdf2image pytesseract")
            raise
    
    def process_pdf(self, file_path: str) -> str:
        """Extract text from PDF files"""
        tracker = LatencyTracker().start()
        
        try:
            elements = self.partition_pdf(file_path)
            text = "\n\n".join([str(element) for element in elements])
            
            tracker.end(f"PDF processing ({os.path.basename(file_path)})")
            return text
        except Exception as e:
            logger.error(f"Error processing PDF {file_path}: {str(e)}")
            raise

##### Text Splitting

In [5]:
class TextSplitter:
    """Handles document chunking using various strategies"""
    
    def __init__(self, chunk_size: int = 512, chunk_overlap: int = 50):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        
        try:
            from langchain_text_splitters import RecursiveCharacterTextSplitter
            self.splitter = RecursiveCharacterTextSplitter(
                chunk_size=self.chunk_size,
                chunk_overlap=self.chunk_overlap,
                separators=["\n\n", "\n", ". ", " ", ""]
            )
        except ImportError:
            logger.error("langchain_text_splitters not installed. Install with: pip install langchain-text-splitters")
            raise
    
    def split_text(self, text: str, metadata: Dict[str, Any] = None) -> List[Dict[str, Any]]:
        """Split text into chunks with metadata"""
        tracker = LatencyTracker().start()
        
        if metadata is None:
            metadata = {}
            
        chunks = self.splitter.create_documents([text], [metadata])
        chunks_with_metadata = []
        
        for i, chunk in enumerate(chunks):
            chunks_with_metadata.append({
                "id": f"{metadata.get('source', 'doc')}_{i}",
                "text": chunk.page_content,
                "metadata": {**chunk.metadata, "chunk_id": i}
            })
        
        tracker.end(f"Text splitting ({len(chunks_with_metadata)} chunks)")
        return chunks_with_metadata

##### EMBEDDING MODEL

In [15]:
class EmbeddingModel:
    """Optimized embedding model with caching and quantization"""
    
    def __init__(self, model_name: str = "BAAI/bge-base-en-v1.5", device: str = None):
        self.model_name = model_name
        
        # Determine device - prefer CUDA, fall back to CPU
        # if device is None:
        #     self.device = "cuda" if torch.cuda.is_available() else "cpu"
        # else:
        self.device = "cpu"
            
        logger.info(f"Using device: {self.device} for embeddings")
        
        try:
            from sentence_transformers import SentenceTransformer
            import torch
            
            tracker = LatencyTracker().start()
            
            # Optimize for CPU inference
            torch.set_num_threads(max(1, multiprocessing.cpu_count() - 1))
            
            # Use smaller model that's faster but still accurate
            # Options: "BAAI/bge-small-en-v1.5" (faster) or "intfloat/e5-small-v2" (very fast)
            self.model = SentenceTransformer(model_name, device=self.device)
            
            # Quantize model for faster inference if on CPU
            if self.device == "cpu":
                try:
                    logger.info("Applying dynamic quantization to embedding model")
                    self.model.model = torch.quantization.quantize_dynamic(
                        self.model.model, {torch.nn.Linear}, dtype=torch.qint8
                    )
                except Exception as e:
                    logger.warning(f"Quantization failed: {e}")
            
            tracker.end(f"Loading optimized embedding model {model_name}")
            
        except ImportError:
            logger.error("sentence_transformers not installed. Install with: pip install sentence-transformers")
            raise
    
    # Add LRU cache to avoid re-computing embeddings for the same text
    @lru_cache(maxsize=1024)
    def _embed_single_text(self, text: str) -> List[float]:
        """Generate embedding for a single text with caching"""
        with torch.no_grad():
            embedding = self.model.encode(text)
            
        if isinstance(embedding, torch.Tensor):
            embedding = embedding.cpu().numpy().tolist()
            
        return embedding
    
    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for a list of texts"""
        tracker = LatencyTracker().start()
        
        # Process in optimal batch size
        batch_size = 8  # Smaller batch size for CPU
        all_embeddings = []
        
        # Use thread pool for parallel processing on CPU
        from concurrent.futures import ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            # Submit tasks in batches
            futures = []
            for i in range(0, len(texts), batch_size):
                batch = texts[i:i+batch_size]
                futures.append(executor.submit(self._batch_embed, batch))
                
            # Collect results
            for future in futures:
                all_embeddings.extend(future.result())
        
        tracker.end(f"Embedding {len(texts)} texts")
        return all_embeddings
    
    def _batch_embed(self, batch: List[str]) -> List[List[float]]:
        """Process a batch of embeddings"""
        with torch.no_grad():
            embeddings = self.model.encode(batch, convert_to_tensor=True)
            embeddings = embeddings.cpu().numpy().tolist()
        return embeddings

    def embed_query(self, query: str) -> List[float]:
        """Generate embedding for a single query"""
        tracker = LatencyTracker().start()
        result = self._embed_single_text(query)
        tracker.end("Query embedding")
        return result

##### VECTOR DATABASE

In [7]:
class VectorStore:
    """Handles vector storage and retrieval using Qdrant"""
    
    def __init__(self, collection_name: str = "documents", vector_size: int = 384):
        self.collection_name = collection_name
        self.vector_size = vector_size
        
        try:
            from qdrant_client import QdrantClient
            from qdrant_client.http import models
            
            # Use local Qdrant instance (can be replaced with cloud URL)
            self.client = QdrantClient(":memory:")  # In-memory for testing; use location="./qdrant_data" for persistence
            
            # Create collection if it doesn't exist
            self.client.recreate_collection(
                collection_name=self.collection_name,
                vectors_config=models.VectorParams(
                    size=self.vector_size,
                    distance=models.Distance.COSINE
                )
            )
            
            logger.info(f"Initialized Qdrant collection: {collection_name}")
            
        except ImportError:
            logger.error("qdrant_client not installed. Install with: pip install qdrant-client")
            raise
    
    def add_documents(self, documents: List[Dict[str, Any]], embeddings: List[List[float]]):
        """Add documents and their embeddings to the vector store"""
        tracker = LatencyTracker().start()
        
        from qdrant_client.http import models
        
        # Prepare points for batch upload
        points = []
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            points.append(models.PointStruct(
                id=i,
                vector=embedding,
                payload={
                    "text": doc["text"],
                    "metadata": doc["metadata"]
                }
            ))
        
        # Upload in batch
        self.client.upsert(
            collection_name=self.collection_name,
            points=points
        )
        
        tracker.end(f"Adding {len(documents)} documents to vector store")
    
    def semantic_search(self, query_embedding: List[float], top_k: int = 5) -> List[Dict[str, Any]]:
        """Retrieve documents using semantic similarity"""
        tracker = LatencyTracker().start()
        
        search_result = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_embedding,
            limit=top_k
        )
        
        results = []
        for res in search_result:
            results.append({
                "id": res.id,
                "text": res.payload["text"],
                "metadata": res.payload["metadata"],
                "score": res.score
            })
        
        tracker.end(f"Semantic search (top-{top_k})")
        return results

##### BM25 LEXICAL SEARCH

In [8]:
class BM25Retriever:
    """Implements BM25 for lexical search"""
    
    def __init__(self):
        try:
            from rank_bm25 import BM25Okapi
            self.BM25Okapi = BM25Okapi
            self.bm25 = None
            self.documents = []
            self.tokenized_corpus = []
        except ImportError:
            logger.error("rank_bm25 not installed. Install with: pip install rank-bm25")
            raise
    
    def add_documents(self, documents: List[Dict[str, Any]]):
        """Index documents for BM25 retrieval"""
        tracker = LatencyTracker().start()
        
        self.documents = documents
        
        # Tokenize and index documents
        import nltk
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt', quiet=True)
        
        from nltk.tokenize import word_tokenize
        
        self.tokenized_corpus = [word_tokenize(doc["text"].lower()) for doc in documents]
        self.bm25 = self.BM25Okapi(self.tokenized_corpus)
        
        tracker.end(f"Indexing {len(documents)} documents for BM25")
    
    def search(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        """Retrieve documents using BM25 lexical similarity"""
        tracker = LatencyTracker().start()
        
        from nltk.tokenize import word_tokenize
        tokenized_query = word_tokenize(query.lower())
        
        # Get BM25 scores
        doc_scores = self.bm25.get_scores(tokenized_query)
        
        # Get top_k documents
        top_indices = doc_scores.argsort()[-top_k:][::-1]
        
        results = []
        for i in top_indices:
            results.append({
                "id": i,
                "text": self.documents[i]["text"],
                "metadata": self.documents[i]["metadata"],
                "score": float(doc_scores[i])  # Convert numpy float to Python float
            })
        
        tracker.end(f"BM25 search (top-{top_k})")
        return results

##### HYBRID RETRIEVAL

In [9]:
class HybridRetriever:
    """Implements hybrid retrieval combining BM25 and embedding search"""
    
    def __init__(
        self, 
        vector_store: VectorStore, 
        bm25_retriever: BM25Retriever,
        embedding_model: EmbeddingModel,
        bm25_weight: float = 0.4,
        semantic_weight: float = 0.6,
    ):
        self.vector_store = vector_store
        self.bm25_retriever = bm25_retriever
        self.embedding_model = embedding_model
        self.bm25_weight = bm25_weight
        self.semantic_weight = semantic_weight
    
    def retrieve(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        """Perform hybrid retrieval"""
        tracker = LatencyTracker().start()
        
        # Get query embedding for semantic search
        query_embedding = self.embedding_model.embed_query(query)
        
        # Retrieve from both sources (get more results than needed for re-ranking)
        retrieval_k = min(top_k * 2, 10)  # Get more docs for re-ranking
        semantic_results = self.vector_store.semantic_search(query_embedding, retrieval_k)
        bm25_results = self.bm25_retriever.search(query, retrieval_k)
        
        # Normalize scores from both retrievers (min-max normalization)
        def normalize_scores(results):
            scores = [r["score"] for r in results]
            if not scores:
                return results
            min_score = min(scores)
            max_score = max(scores)
            score_range = max_score - min_score
            
            if score_range > 0:
                for r in results:
                    r["normalized_score"] = (r["score"] - min_score) / score_range
            else:
                for r in results:
                    r["normalized_score"] = 1.0
                    
            return results
        
        semantic_results = normalize_scores(semantic_results)
        bm25_results = normalize_scores(bm25_results)
        
        # Create a unified document pool with combined scores
        doc_map = {}
        
        # Add semantic results
        for doc in semantic_results:
            doc_id = doc["id"]
            doc_map[doc_id] = {
                "id": doc_id,
                "text": doc["text"],
                "metadata": doc["metadata"],
                "semantic_score": doc["normalized_score"] * self.semantic_weight,
                "bm25_score": 0,  # Will be updated if in BM25 results
                "combined_score": doc["normalized_score"] * self.semantic_weight
            }
        
        # Add/update with BM25 results
        for doc in bm25_results:
            doc_id = doc["id"]
            if doc_id in doc_map:
                # Update existing entry
                doc_map[doc_id]["bm25_score"] = doc["normalized_score"] * self.bm25_weight
                doc_map[doc_id]["combined_score"] += doc["normalized_score"] * self.bm25_weight
            else:
                # New entry
                doc_map[doc_id] = {
                    "id": doc_id,
                    "text": doc["text"],
                    "metadata": doc["metadata"],
                    "semantic_score": 0,  # Not in semantic results
                    "bm25_score": doc["normalized_score"] * self.bm25_weight,
                    "combined_score": doc["normalized_score"] * self.bm25_weight
                }
        
        # Sort by combined score and take top_k
        ranked_results = sorted(
            doc_map.values(), 
            key=lambda x: x["combined_score"], 
            reverse=True
        )[:top_k]
        
        tracker.end(f"Hybrid retrieval (top-{top_k})")
        return ranked_results

##### LLM INTERFACE

In [11]:
class LLMInterface:
    """Optimized interface for interacting with LLMs"""
    
    def __init__(self, model_name: str = "mistral-7b-instruct-v0.2.Q3_K_S.gguf"):
        self.model_name = model_name
        
        # Determine if we're using a local GGUF model or an API
        if model_name.endswith(".gguf"):
            self._init_local_llm(model_name)
        else:
            raise ValueError("Currently only supporting local GGUF models. Use a .gguf file.")
    
    def _init_local_llm(self, model_name: str):
        """Initialize a local LLM using llama.cpp Python bindings with optimizations"""
        try:
            from llama_cpp import Llama
            
            # Check if model exists in the models directory
            model_dir = os.path.join(os.getcwd(), "models")
            model_path = os.path.join(model_dir, model_name)
            
            if not os.path.exists(model_path):
                logger.warning(f"Model file not found at {model_path}. Please download it first.")
                logger.info(f"You can download Mistral 7B Instruct GGUF from: https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF")
                model_path = model_name  # Try with just the name in case it's in PATH
            
            tracker = LatencyTracker().start()
            
            # Get system memory information
            mem_info = psutil.virtual_memory()
            available_ram_gb = mem_info.available / (1024**3)
            
            # Calculate optimal thread count based on CPU cores
            # Leave 1-2 cores for the OS
            physical_cores = psutil.cpu_count(logical=False) or 4
            n_threads = max(1, physical_cores - 1)
            
            # Determine suitable model layers to GPU
            n_gpu_layers = 0  # Keep at 0 for CPU only
            
            # Load model with optimized parameters for CPU inference
            # Use more aggressive quantization (Q3_K_S or Q2_K) for faster inference
            self.llm = Llama(
                model_path=model_path,
                n_ctx=2048,          # Reduced context window to save memory
                n_threads=n_threads,
                n_batch=512,         # Batch size for prompt processing
                n_gpu_layers=n_gpu_layers,
                use_mlock=True,      # Keep model in RAM
                use_mmap=True,       # Use memory mapping for faster loading
                vocab_only=False,    # Load the full model
                seed=-1              # Random seed for reproducibility
            )
            
            logger.info(f"LLM initialized with {n_threads} threads, context size: 2048")
            tracker.end(f"Loading optimized LLM model {model_name}")
            
        except ImportError:
            logger.error("llama-cpp-python not installed. Install with: pip install llama-cpp-python")
            raise
    
    @lru_cache(maxsize=128)
    def generate_standalone_question(self, current_question: str, chat_history_key: str) -> str:
        """Generate a standalone question with caching"""
        tracker = LatencyTracker().start()
        
        # Parse chat history from key (needed for caching)
        # Format: last 3 turns max, 100 chars max per turn
        chat_history = []
        turns = chat_history_key.split("|||")
        for turn in turns:
            if not turn:
                continue
            parts = turn.split(":::")
            if len(parts) == 2:
                chat_history.append({"user": parts[0], "assistant": parts[1]})
        
        # Format chat history for context - use minimal formatting
        history_text = ""
        for turn in chat_history[-2:]:  # Just use last 2 turns for efficiency
            if "user" in turn:
                history_text += f"User: {turn['user']}\n"
            if "assistant" in turn:
                history_text += f"Assistant: {turn['assistant']}\n"
        
        # Simplified prompt
        prompt = f"""<s>[INST] Based on this conversation:
{history_text}
Rewrite this follow-up question as a standalone question: {current_question} [/INST]"""
        
        # Generate response with optimized parameters
        response = self.llm(
            prompt,
            max_tokens=100,  # Reduced max tokens
            temperature=0.0,  # Deterministic output
            stop=["</s>"]
        )
        
        standalone_question = response["choices"][0]["text"].strip()
        
        # If the standalone question is too similar to the original, just use original
        if len(standalone_question) < 5 or standalone_question == current_question:
            standalone_question = current_question
        
        tracker.end("Generating standalone question")
        logger.info(f"Follow-up: '{current_question}' -> Standalone: '{standalone_question}'")
        
        return standalone_question
    
    def generate_answer(self, query: str, retrieved_docs: List[Dict[str, Any]]) -> str:
        """Generate an answer with optimized prompt"""
        tracker = LatencyTracker().start()
        
        # Format retrieved documents - only use top 3 and limit text length
        context = ""
        for i, doc in enumerate(retrieved_docs[:3], 1):
            source = doc["metadata"].get("source", "Unknown")
            # Limit each doc to ~500 chars to reduce context size
            doc_text = doc["text"]
            if len(doc_text) > 500:
                doc_text = doc_text[:497] + "..."
            context += f"\nDoc {i} [{source}]: {doc_text}\n"
        
        # Create minimal prompt 
        prompt = f"""<s>[INST] Question: {query}

Context:
{context}

Answer the question based only on the provided context. Be concise. [/INST]"""
        
        # Generate response with optimized parameters
        response = self.llm(
            prompt,
            max_tokens=256,     # Reduced max tokens
            temperature=0.1,    # Slight randomness
            stop=["</s>"],
            top_p=0.9,          # More focused sampling
            top_k=40            # Limit vocabulary choices
        )
        
        answer = response["choices"][0]["text"].strip()
        
        tracker.end("Generating answer")
        return answer

##### RAG SYSTEM

In [12]:
class RAGSystem:
    """Main RAG system that orchestrates the entire process"""
    
    def __init__(self, top_k: int = 3):
        self.top_k = top_k
        
        # Initialize components
        self.document_processor = DocumentProcessor()
        self.text_splitter = TextSplitter(chunk_size=512, chunk_overlap=50)
        self.embedding_model = EmbeddingModel(model_name="BAAI/bge-small-en-v1.5")
        self.vector_store = VectorStore(
            collection_name="documents",
            vector_size=384  # BGE-small embedding dimension
        )
        self.bm25_retriever = BM25Retriever()
        self.hybrid_retriever = HybridRetriever(
            vector_store=self.vector_store,
            bm25_retriever=self.bm25_retriever,
            embedding_model=self.embedding_model,
            bm25_weight=0.4,
            semantic_weight=0.6,
        )
        self.llm = LLMInterface(model_name="mistral-7b-instruct-v0.2.Q4_K_M.gguf")
        
        # Chat history for conversational context
        self.chat_history = []
    
    def ingest_documents(self, file_paths: List[str]):
        """Process and index documents"""
        total_tracker = LatencyTracker().start()
        all_chunks = []
        
        # Process each document
        for file_path in file_paths:
            if file_path.lower().endswith(".pdf"):
                text = self.document_processor.process_pdf(file_path)
                chunks = self.text_splitter.split_text(
                    text,
                    metadata={"source": os.path.basename(file_path)}
                )
                all_chunks.extend(chunks)
            else:
                logger.warning(f"Unsupported file type: {file_path}")
        
        # Extract texts for embedding
        texts = [chunk["text"] for chunk in all_chunks]
        
        # Generate embeddings
        embeddings = self.embedding_model.embed_texts(texts)
        
        # Index in both vector store and BM25
        self.vector_store.add_documents(all_chunks, embeddings)
        self.bm25_retriever.add_documents(all_chunks)
        
        total_tracker.end(f"Ingesting {len(file_paths)} documents ({len(all_chunks)} chunks)")
        return len(all_chunks)
    
    def query(self, question: str) -> Dict[str, Any]:
        """Process a query and return results"""
        total_tracker = LatencyTracker().start()
        
        # Handle follow-up questions
        if self.chat_history:
            # Create a hashable key for the chat history (for caching)
            history_key = "|||".join([
                f"{turn.get('user', '')}:::{turn.get('assistant', '')}" 
                for turn in self.chat_history[-3:]  # Only use last 3 turns
            ])
            standalone_question = self.llm.generate_standalone_question(
                question, 
                history_key
            )
        else:
            standalone_question = question
        
        # Retrieve relevant documents
        retrieved_docs = self.hybrid_retriever.retrieve(
            standalone_question, 
            top_k=self.top_k
        )
        
        # Generate answer
        answer = self.llm.generate_answer(standalone_question, retrieved_docs)
        
        # Update chat history
        self.chat_history.append({"user": question})
        self.chat_history.append({"assistant": answer})

        # Limit chat history length to prevent memory growth
        if len(self.chat_history) > 10:
            self.chat_history = self.chat_history[-10:]
        
        # Prepare sources information for citation
        sources = []
        for doc in retrieved_docs:
            source = doc["metadata"].get("source", "Unknown")
            if source not in [s["name"] for s in sources]:
                sources.append({
                    "name": source,
                    "score": doc["combined_score"]
                })
        
        result = {
            "question": question,
            "standalone_question": standalone_question,
            "answer": answer,
            "sources": sources,
            "latency": total_tracker.end("Total query processing")
        }
        
        return result

##### EVALUATION METRICS

In [13]:
class RAGEvaluator:
    """Evaluates RAG system performance"""
    
    def __init__(self, rag_system: RAGSystem):
        self.rag_system = rag_system
        
        try:
            import nltk
            try:
                nltk.data.find('tokenizers/punkt')
                nltk.data.find('metrics/bleu_ref.txt')
            except LookupError:
                nltk.download('punkt', quiet=True)
                nltk.download('bleu_ref', quiet=True)
                
            import rouge
            self.rouge = rouge.Rouge()
            
        except ImportError:
            logger.error("Evaluation packages not installed. Install with: pip install nltk rouge-score")
            raise
    
    def evaluate_retrieval(self, query: str, relevant_doc_ids: List[str], retrieved_docs: List[Dict[str, Any]]) -> Dict[str, float]:
        """Evaluate retrieval performance using precision, recall, f1"""
        retrieved_ids = [str(doc["id"]) for doc in retrieved_docs]
        
        # Calculate metrics
        true_positives = len(set(retrieved_ids).intersection(set(relevant_doc_ids)))
        precision = true_positives / len(retrieved_ids) if retrieved_ids else 0
        recall = true_positives / len(relevant_doc_ids) if relevant_doc_ids else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        return {
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
        }
    
    def evaluate_generation(self, generated_answer: str, reference_answer: str) -> Dict[str, float]:
        """Evaluate answer generation using BLEU and ROUGE"""
        import nltk
        from nltk.translate.bleu_score import sentence_bleu
        
        # Prepare for BLEU
        reference_tokens = [nltk.word_tokenize(reference_answer.lower())]
        generated_tokens = nltk.word_tokenize(generated_answer.lower())
        
        # Calculate BLEU score
        bleu_score = sentence_bleu(reference_tokens, generated_tokens)
        
        # Calculate ROUGE scores
        try:
            rouge_scores = self.rouge.get_scores(generated_answer, reference_answer)[0]
        except:
            rouge_scores = {"rouge-1": {"f": 0}, "rouge-2": {"f": 0}, "rouge-l": {"f": 0}}
        
        return {
            "bleu": bleu_score,
            "rouge1_f1": rouge_scores["rouge-1"]["f"],
            "rouge2_f1": rouge_scores["rouge-2"]["f"],
            "rougeL_f1": rouge_scores["rouge-l"]["f"],
        }
    
    def evaluate_latency(self, result: Dict[str, Any]) -> Dict[str, float]:
        """Evaluate system latency"""
        return {
            "total_latency": result["latency"],
        }
    
    def run_benchmark(self, test_queries: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Run a benchmark on a set of test queries with reference answers"""
        results = {
            "retrieval": {
                "precision": [],
                "recall": [],
                "f1_score": [],
            },
            "generation": {
                "bleu": [],
                "rouge1_f1": [],
                "rouge2_f1": [],
                "rougeL_f1": [],
            },
            "latency": {
                "total_latency": [],
            }
        }
        
        for test in tqdm(test_queries, desc="Running benchmark"):
            query = test["query"]
            reference_answer = test["reference_answer"]
            relevant_doc_ids = test.get("relevant_doc_ids", [])
            
            # Run query
            result = self.rag_system.query(query)
            generated_answer = result["answer"]
            
            # We need to get the retrieved docs from the result for evaluation
            # This would require modifying the RAG system to return retrieved docs
            # For this example, we'll just use an empty list
            retrieved_docs = []  # Placeholder
            
            # Evaluate
            retrieval_metrics = self.evaluate_retrieval(query, relevant_doc_ids, retrieved_docs)
            generation_metrics = self.evaluate_generation(generated_answer, reference_answer)
            latency_metrics = self.evaluate_latency(result)
            
            # Collect results
            for k, v in retrieval_metrics.items():
                results["retrieval"][k].append(v)
            for k, v in generation_metrics.items():
                results["generation"][k].append(v)
            for k, v in latency_metrics.items():
                results["latency"][k].append(v)
        
        # Calculate averages
        summary = {
            "retrieval": {},
            "generation": {},
            "latency": {}
        }
        
        for category in results:
            for metric in results[category]:
                values = results[category][metric]
                summary[category][metric] = sum(values) / len(values) if values else 0
        
        return {
            "details": results,
            "summary": summary
        }

#### Main

In [16]:
def main():
    """Example usage of the RAG system"""
    
    # Initialize the system
    rag = RAGSystem(top_k=3)
    
    # Ingest documents (example paths)
    document_paths = [
        "Documents/EveriseHandbook.pdf",
    ]
    
    # Check if documents exist
    existing_docs = [path for path in document_paths if os.path.exists(path)]
    if not existing_docs:
        logger.warning(f"No documents found at the specified paths. Skipping ingestion.")
    else:
        rag.ingest_documents(existing_docs)
    
    # Interactive querying
    print("\n===== RAG Conversational System Demo =====")
    print("Type 'exit' to quit\n")
    
    while True:
        question = input("\nYour question: ")
        if question.lower() in ["exit", "quit"]:
            break
            
        print("\nProcessing...")
        
        try:
            result = rag.query(question)
            
            print(f"\nAnswer: {result['answer']}")
            print(f"\nSources:")
            for source in result['sources']:
                print(f"- {source['name']} (score: {source['score']:.3f})")
            print(f"\nLatency: {result['latency']:.3f} seconds")
            
        except Exception as e:
            logger.error(f"Error processing query: {str(e)}")
            print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

2025-03-03 15:00:11,676 - rag_system - INFO - Using device: cpu for embeddings
2025-03-03 15:00:43,908 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2025-03-03 15:00:49,997 - rag_system - INFO - Applying dynamic quantization to embedding model
2025-03-03 15:00:49,999 - rag_system - WARNING - Quantization failed: 'SentenceTransformer' object has no attribute 'model'
2025-03-03 15:00:49,999 - rag_system - INFO - LATENCY: Loading optimized embedding model BAAI/bge-small-en-v1.5 took 6.1812 seconds
C:\Users\Magrawal\AppData\Local\Temp\ipykernel_19980\1932192397.py:16: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  self.client.recreate_collection(
2025-03-03 15:00:51,767 - rag_system - INFO - Initialized Qdrant collection: documents
llama_model_loader: loaded meta data with 24 key-value 


===== RAG Conversational System Demo =====
Type 'exit' to quit




Your question:  tell me about the policies



Processing...


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.91it/s]
2025-03-03 15:08:40,242 - rag_system - INFO - LATENCY: Query embedding took 0.1123 seconds
C:\Users\Magrawal\AppData\Local\Temp\ipykernel_19980\1932192397.py:60: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self.client.search(
2025-03-03 15:08:40,262 - rag_system - INFO - LATENCY: Semantic search (top-6) took 0.0200 seconds
2025-03-03 15:08:40,262 - rag_system - INFO - LATENCY: BM25 search (top-6) took 0.0000 seconds
2025-03-03 15:08:40,262 - rag_system - INFO - LATENCY: Hybrid retrieval (top-3) took 0.1322 seconds
C:\Users\Magrawal\Downloads\Experiments\Humana-poc\humanaVenv\lib\site-packages\llama_cpp\llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
llama_perf_context_print:


Answer: The Conflict of Interest Policy is a document outlining the guidelines for Everise employees to identify and manage potential conflicts of interest between their personal interests and their duties at the company. The policy is subject to change and employees are encouraged to seek clarification from the Chief Legal Officer if they have any questions.

Sources:
- EveriseHandbook.pdf (score: 0.600)

Latency: 79.039 seconds



Your question:  exit


In [19]:
"""
Self-Evaluation Module for RAG System
- Automatically generates test queries
- Creates synthetic reference answers
- Evaluates system without external data
"""

import json
import time
import random
from typing import List, Dict, Any
import logging

logger = logging.getLogger("rag_system")

class RAGSelfEvaluator:
    """Self-evaluation for RAG systems without external datasets"""
    
    def __init__(self, rag_system):
        self.rag_system = rag_system
        
        try:
            import nltk
            try:
                nltk.data.find('tokenizers/punkt')
                nltk.data.find('metrics/bleu_ref.txt')
            except LookupError:
                nltk.download('punkt', quiet=True)
                nltk.download('bleu_ref', quiet=True)
                
            import rouge
            self.rouge = rouge.Rouge()
            
        except ImportError:
            logger.error("Evaluation packages not installed. Install with: pip install nltk rouge-score")
            raise
    
    def generate_test_queries(self, num_queries: int = 5) -> List[Dict[str, Any]]:
        """Generate test queries based on documents in the system"""
        # Get all documents from the system
        # This assumes we can access the documents - we'll extract from BM25 retriever
        if not hasattr(self.rag_system, 'bm25_retriever') or not self.rag_system.bm25_retriever.documents:
            logger.warning("No documents found in the system for query generation")
            return []
        
        all_docs = self.rag_system.bm25_retriever.documents
        if not all_docs:
            logger.warning("No documents found in the system for query generation")
            return []
            
        test_queries = []
        
        # Extract key sentences from random documents
        for _ in range(num_queries):
            # Pick a random document
            doc = random.choice(all_docs)
            doc_text = doc["text"]
            
            # Split into sentences
            import nltk
            sentences = nltk.sent_tokenize(doc_text)
            
            if not sentences:
                continue
                
            # Pick a random sentence as the base for our query
            sentence = random.choice(sentences)
            
            # Create a question from the sentence
            question_types = [
                "What is ", "How does ", "Can you explain ", 
                "Tell me about ", "Why is ", "What are "
            ]
            
            # Extract key terms (nouns) from the sentence
            words = nltk.word_tokenize(sentence)
            tagged = nltk.pos_tag(words)
            
            # Find nouns
            nouns = [word for word, tag in tagged if tag.startswith('NN')]
            
            if not nouns:
                # If no nouns, just use the whole sentence
                subject = sentence
            else:
                # Use a random noun as the subject
                subject = random.choice(nouns)
            
            # Generate the question
            question = random.choice(question_types) + subject + "?"
            
            # The source document will be our "relevant" document
            relevant_doc_id = doc["id"]
            
            # Use the sentence as our "reference answer"
            reference_answer = sentence
            
            test_queries.append({
                "query": question,
                "relevant_doc_ids": [str(relevant_doc_id)],
                "reference_answer": reference_answer,
                "source_doc": doc["id"],
                "source_text": doc_text[:200] + "..." # First 200 chars for reference
            })
        
        return test_queries
    
    def evaluate_query(self, query: Dict[str, Any]) -> Dict[str, Any]:
        """Evaluate a single query"""
        start_time = time.time()
        
        # Run the query
        result = self.rag_system.query(query["query"])
        
        # Calculate latency
        latency = time.time() - start_time
        
        # Calculate ROUGE scores
        try:
            rouge_scores = self.rouge.get_scores(result["answer"], query["reference_answer"])[0]
        except:
            rouge_scores = {"rouge-1": {"f": 0}, "rouge-2": {"f": 0}, "rouge-l": {"f": 0}}
        
        # Check if source document was retrieved
        # We can't directly check this without modifying the RAG system
        # So we'll check if any key phrases from the source appear in the answer
        source_text = query["source_text"]
        key_phrases = nltk.word_tokenize(source_text)[:20]  # First 20 tokens
        
        # Count how many key phrases appear in the answer
        answer_tokens = nltk.word_tokenize(result["answer"].lower())
        key_phrase_overlap = sum(1 for token in key_phrases if token.lower() in answer_tokens)
        retrieval_score = key_phrase_overlap / max(1, len(key_phrases))
        
        evaluation = {
            "query": query["query"],
            "reference_answer": query["reference_answer"],
            "generated_answer": result["answer"],
            "latency": latency,
            "rouge1_f1": rouge_scores["rouge-1"]["f"],
            "rouge2_f1": rouge_scores["rouge-2"]["f"], 
            "rougeL_f1": rouge_scores["rouge-l"]["f"],
            "retrieval_score": retrieval_score,
            "sources": result.get("sources", [])
        }
        
        return evaluation
    
    def run_self_evaluation(self, num_queries: int = 5) -> Dict[str, Any]:
        """Run a complete self-evaluation"""
        # Generate test queries
        logger.info(f"Generating {num_queries} test queries")
        test_queries = self.generate_test_queries(num_queries)
        
        if not test_queries:
            return {"error": "Could not generate test queries. No documents available."}
        
        # Evaluate each query
        results = []
        for query in test_queries:
            logger.info(f"Evaluating query: {query['query']}")
            result = self.evaluate_query(query)
            results.append(result)
        
        # Calculate average scores
        avg_latency = sum(r["latency"] for r in results) / len(results)
        avg_rouge1 = sum(r["rouge1_f1"] for r in results) / len(results)
        avg_rouge2 = sum(r["rouge2_f1"] for r in results) / len(results)
        avg_rougeL = sum(r["rougeL_f1"] for r in results) / len(results)
        avg_retrieval = sum(r["retrieval_score"] for r in results) / len(results)
        
        summary = {
            "num_queries": len(results),
            "avg_latency": avg_latency,
            "avg_rouge1_f1": avg_rouge1,
            "avg_rouge2_f1": avg_rouge2,
            "avg_rougeL_f1": avg_rougeL,
            "avg_retrieval_score": avg_retrieval,
            "results": results
        }
        
        # Log summary
        logger.info("Evaluation Summary:")
        logger.info(f"  Number of Queries: {summary['num_queries']}")
        logger.info(f"  Average Latency: {summary['avg_latency']:.2f} seconds")
        logger.info(f"  Average ROUGE-1 F1: {summary['avg_rouge1_f1']:.4f}")
        logger.info(f"  Average ROUGE-L F1: {summary['avg_rougeL_f1']:.4f}")
        logger.info(f"  Average Retrieval Score: {summary['avg_retrieval_score']:.4f}")
        
        return summary